In [1]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
from libsvm.svmutil import *
from scipy.spatial.distance import cdist

# SVM Task-1

## MNIST Reading

X_train.csv is a $5000 \times 784 $matrix. Every row corresponds to a $28 \times 28 $gray-scale image. <br>
Y_train.csv is a $5000 \times 1 $matrix, which records the class of the training samples.<br>

In [2]:
# for each row in X_train.csv is an image
def load_X_data(file): 
    X=[]
    with open(file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            X.append(np.array(row))
        X = np.asarray(X, dtype=np.float32)
    return X

# 5000 class category
def load_y_data(file):
    y=[]
    with open(file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            y.append(row[0])
        y = np.asarray(y, dtype=int)
    return y

In [3]:
train_X = load_X_data(r'data\ML_HW05\data\X_train.csv')
train_y = load_y_data(r'data\ML_HW05\data\y_train.csv')
test_X = load_X_data(r'data\ML_HW05\data\X_test.csv')
test_y = load_y_data(r'data\ML_HW05\data\y_test.csv')

## API svm_train param

-t kernel_type : set type of kernel function (default 2) <br>
&emsp;4 -- precomputed kernel (kernel values in training_set_file)<br>
 

In [4]:
kernel_type = ['-t 0', '-t 1', '-t 1']

## Linear Kernel

$k(X_{i}, X) = X_{i} \cdot X$

In [5]:
def _Linear_kernel(xa, xb):
    return xa@xb.T

## RBF Kernel

$K(x,x') = exp(-\gamma ||x-x'||_{2}^{2})$

In [6]:
def _RBF_kernel(xa, xb, gamma):
    return np.exp(- gamma * cdist(xa, xb, 'sqeuclidean'))

In [7]:
def precompute_kernel(xa, xb):
    linear_kernel = _Linear_kernel(xa, xb)
    RBF_kernel = _RBF_kernel(xa, xb, 0.01)
    cus_kernel = linear_kernel + RBF_kernel
    cus_kernel = np.hstack((np.arange(1,len(xa)+1).reshape(-1,1),cus_kernel))
    
    return cus_kernel

In [8]:
cus_train_kernel = precompute_kernel(train_X, train_X)
cus_test_kernel = precompute_kernel(test_X, train_X)

prob = svm_problem(train_y, cus_train_kernel, isKernel=True)
param = svm_parameter('-q -t 4')
model = svm_train(prob, param)
p_label, p_acc, p_vals = svm_predict(test_y, cus_test_kernel, model, '-q')
print('linear kernel + RBF kernel accuracy: {:.2f}%'.format(p_acc[0]))

linear kernel + RBF kernel accuracy: 95.32%
